In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import math,os
from numpy.random import choice
import scikitplot as skplt
from time import time

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
pd.options.display.max_rows = 100
pd.set_option('display.max_columns', None)

In [4]:
from sklearn.linear_model import LinearRegression,Ridge,SGDRegressor,ElasticNet
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.utils.validation import check_array 
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.svm import LinearSVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,ExtraTreesRegressor,GradientBoostingRegressor,AdaBoostRegressor,BaggingRegressor

In [5]:
historicalColumns,neighborColumns,neighborColumnsAggregated = [],[],[]

for historical in range(5):
    historicalColumns += ['Tminus'+str(historical+1)]

for neighbor in range(26):
    neighborColumns += ['T'+str(neighbor+1)+'_t-1']
    
for neighborDegree in range(3):
        neighborColumnsAggregated += ['T_nbhDeg'+str(neighborDegree+1)+'_t-1']

columns = ['voxelLat','voxelLong','voxelVert','voxelType','timestep','x_voxel','y_voxel','z_voxel','layerNum','time_creation', 'time_elapsed', 'x_laser','y_laser','z_laser','x_distance','y_distance','z_distance','euclidean_distance_laser'] + historicalColumns+ neighborColumns + neighborColumnsAggregated + ['T_self']

In [6]:
def roundup(a, digits=4):
    n = 10**-digits
    return round(math.ceil(a / n) * n, digits)

def isEven(num):
    if num%2 ==0:
        return True
    return False

def modLog(num):
    try:
        return log(num)
    except:
        return 0

def loadNumpy(name,path='.'):
    if ".npy" in name:
        fullPath = path+'/'+name
    else:
        fullPath = path+'/'+name+'.npy'
    return np.load(fullPath, allow_pickle=True)

In [7]:
featureColumns = ['timestep','x_distance','y_distance','z_distance','time_elapsed'] + historicalColumns + neighborColumns
featureDisplay = featureColumns

# featureDisplay[7] = 'T_immediate_x-1'
# featureDisplay[8] = 'T_immediate_x+1'
# featureDisplay[9] = 'T_immediate_y-1'
# featureDisplay[10] = 'T_immediate_y+1'
# featureDisplay[11] = 'T_immediate_z-1'

# featureDisplay[13] = 'T_immediate_x-1,y-1'
# featureDisplay[14] = 'T_immediate_x-1,y+1'
# featureDisplay[15] = 'T_immediate_x+1,y-1'
# featureDisplay[16] = 'T_immediate_x+1,y+1'

# featureDisplay[21] = 'T_immediate_x-1,z-1'

In [8]:
def plot_feature_importances(et):
    skplt.estimators.plot_feature_importances(et,text_fontsize=16,max_num_features=6,figsize=(24,4),feature_names=featureDisplay)

In [9]:
def mean_absolute_percentage_error(y_true, y_pred):
	'''
	scikit(sklearn) does not have support for mean absolute percentage error MAPE.
	This is because the denominator can theoretically be 0 and so the value would be undefined.
	So this is our implementation
	'''
# 	y_true = check_array(y_true)
# 	y_pred = check_array(y_pred)

	return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def r2(y_true,y_pred):
    return roundup(r2_score(y_true,y_pred))

def mse(y_true,y_pred):
    return roundup(mean_squared_error(y_true,y_pred))

def mae(y_true,y_pred):
    return roundup(mean_absolute_error(y_true,y_pred))

def mape(y_true, y_pred):
    return roundup(mean_absolute_percentage_error(y_true,y_pred))

In [10]:
def combineDataFrames(prefix,columns=columns):
    List = []
    nums_start,nums_stop = [],[]
    for item in os.listdir('../data/cube-20-20-10-800-processed'):
        if "cubeAgg-20-20-10-800_" in item and ".npy" in item:
            timeStep_start = int(item.split('cubeAgg-20-20-10-800_')[1].split('_')[0])
            nums_start += [timeStep_start]
            
            timeStep_stop = int(item.split('_')[2].split('.npy')[0])
            nums_stop += [timeStep_stop]
            
    nums_start = sorted(nums_start)
    nums_stop = sorted(nums_stop)
    
#     print (nums_start)
#     print (nums_stop)
    
    array = loadNumpy('../data/cube-20-20-10-800-processed/'+prefix+'_'+str(nums_start[0])+'_'+str(nums_stop[0])+'.npy')
    for i in range(1,len(nums_start)):
        newFile = '../data/cube-20-20-10-800-processed/'+prefix+'_'+str(nums_start[i])+'_'+str(nums_stop[i])+'.npy'
        array = np.append(array,loadNumpy(newFile),axis=0)
    return pd.DataFrame(array,columns=columns)


# def combineDataFrames(columns=columns):
#     dir = './temp/'
#     array = loadNumpy(dir+'file1.npy')
#     array = np.append(array,loadNumpy(dir+'file2.npy'),axis=0)
    
#     return pd.DataFrame(array,columns=columns)

In [11]:
df_big = combineDataFrames('cubeAgg-20-20-10-800')

## Iterative Prediction (Trail Run)

In [ ]:
df_train_1 = df_big[df_big.timestep < 200.0]
df_test_1 = df_big[(df_big.timestep >= 200.0) & (df_big.timestep < 220.0)]

# featureColumns = ['timestep','x_distance','y_distance','z_distance','layerNum','Tminus1','Tminus2']+neighborColumns

X_train_1, y_train_1= shuffle(df_train_1.loc[:,featureColumns ], df_train_1['T_self'].values, random_state=300)
X_test_1,y_test_1 = shuffle(df_test_1.loc[:,featureColumns],df_test_1['T_self'],random_state=300)

et_1 = ExtraTreesRegressor(n_estimators=10, n_jobs=-1,random_state=300)
et_1.fit(X_train_1,y_train_1)
y_predicted_1 = et_1.predict(X_test_1)
print('Iteration 1 over....')

X_train_2, y_train_2= X_train_1.append(X_test_1, ignore_index=True), np.append( y_train_1, y_predicted_1)
X_train_2,y_train_2 = shuffle(X_train_2,y_train_2,random_state=300)

df_test_2 = df_big[(df_big.timestep >= 220.0) & (df_big.timestep < 240.0)]
X_test_2,y_test_2 = shuffle(df_test_2.loc[:,featureColumns],df_test_2['T_self'],random_state=300)

et_2 = ExtraTreesRegressor(n_estimators=10, n_jobs=-1,random_state=300)
et_2.fit(X_train_2,y_train_2)
y_predicted_2 = et_2.predict(X_test_2)
print('Iteration 2 over....')

X_train_3, y_train_3= X_train_2.append(X_test_2, ignore_index=True), np.append( y_train_2, y_predicted_2)
X_train_3,y_train_3 = shuffle(X_train_3,y_train_3,random_state=300)

df_test_3 = df_big[(df_big.timestep >= 240.0) & (df_big.timestep < 260.0)]
X_test_3,y_test_3 = shuffle(df_test_3.loc[:,featureColumns],df_test_3['T_self'],random_state=300)

et_3 = ExtraTreesRegressor(n_estimators=10, n_jobs=-1,random_state=300)
et_3.fit(X_train_3,y_train_3)
y_predicted_3 = et_3.predict(X_test_3)
print('Iteration 3 over....')

X_train_4, y_train_4= X_train_3.append(X_test_3, ignore_index=True), np.append( y_train_3, y_predicted_3)
X_train_4,y_train_4 = shuffle(X_train_4,y_train_4,random_state=300)

df_test_4 = df_big[(df_big.timestep >= 260.0) & (df_big.timestep < 280.0)]
X_test_4,y_test_4 = shuffle(df_test_4.loc[:,featureColumns],df_test_4['T_self'],random_state=300)

et_4 = ExtraTreesRegressor(n_estimators=10, n_jobs=-1,random_state=300)
et_4.fit(X_train_4,y_train_4)
y_predicted_4 = et_4.predict(X_test_4)
print('Iteration 4 over....')

X_train_5, y_train_5= X_train_4.append(X_test_4, ignore_index=True), np.append( y_train_4, y_predicted_4)
X_train_5,y_train_5 = shuffle(X_train_5,y_train_5,random_state=300)

df_test_5 = df_big[(df_big.timestep >= 280.0) & (df_big.timestep < 300.0)]
X_test_5,y_test_5 = shuffle(df_test_5.loc[:,featureColumns],df_test_5['T_self'],random_state=300)

et_5 = ExtraTreesRegressor(n_estimators=10, n_jobs=-1,random_state=300)
et_5.fit(X_train_5,y_train_5)
y_predicted_5 = et_5.predict(X_test_5)
print('Iteration 5 over....')


print ('Iterative training results')
print r2(y_test_1,y_predicted_1), mape(y_test_1,y_predicted_1)
print r2(y_test_2,y_predicted_2), mape(y_test_2,y_predicted_2)
print r2(y_test_3,y_predicted_3), mape(y_test_3,y_predicted_3)
print r2(y_test_4,y_predicted_4), mape(y_test_4,y_predicted_4)
print r2(y_test_5,y_predicted_5), mape(y_test_5,y_predicted_5)

print ('One step training results')
et_direct = ExtraTreesRegressor(n_estimators=10, n_jobs=-1,random_state=300)
et_direct.fit(X_train_1,y_train_1)
y_predicted = et_direct.predict(X_test_5)

r2(y_test_5,y_predicted) ,mape(y_test_5,y_predicted)

## Iterative vs Non-iterative prediction

In [12]:
def compare_iterative_direct_prediction(n=5, init=100, TIMESTEP_ITER = 50, n_estimators=10):
    
    df_train_i_minus_1 = df_big[df_big.timestep < init]
    df_test_i_minus_1 = df_big[(df_big.timestep >= init) & (df_big.timestep < init+TIMESTEP_ITER)]
#     print (df_test_i_minus_1)

    X_train_i_minus_1, y_train_i_minus_1= shuffle(df_train_i_minus_1.loc[:,featureColumns ], df_train_i_minus_1['T_self'].values, random_state=300)
    X_test_i_minus_1,y_test_i_minus_1 = shuffle(df_test_i_minus_1.loc[:,featureColumns],df_test_i_minus_1['T_self'],random_state=300)

    et_i_minus_1 = ExtraTreesRegressor(n_estimators=n_estimators, n_jobs=-1,random_state=300)
    start = time()
    et_i_minus_1.fit(X_train_i_minus_1,y_train_i_minus_1)
    y_predicted_i_minus_1 = et_i_minus_1.predict(X_test_i_minus_1)
    
    START = init
    STOP = init + TIMESTEP_ITER
    
#     print ("START is: ",START)
#     print ("STOP is: ", STOP)
    
#     print ("r2 is: ",r2(y_test_i_minus_1,y_predicted_i_minus_1) , "mape is: ", mape(y_test_i_minus_1,y_predicted_i_minus_1))
    
    temp_y = y_test_i_minus_1
    temp_predicted = y_predicted_i_minus_1

    print('Iteration 1 over....')
#     print('\n')
    
    
    for i in range(2,n+1):
        X_train_i, y_train_i= X_train_i_minus_1.append(X_test_i_minus_1, ignore_index=True), np.append(y_train_i_minus_1, y_predicted_i_minus_1)
#         print(X_train_i.iloc[:,0:2].tail(50))
#         print(X_train_i_minus_1.iloc[:,0:2].tail(50))
        X_train_i,y_train_i = shuffle(X_train_i,y_train_i,random_state=300)
        START = START + TIMESTEP_ITER
        STOP = STOP + TIMESTEP_ITER
#         print ("START is: ",START)
#         print ("STOP is: ", STOP)
        df_test_i = df_big[(df_big.timestep >= START) & (df_big.timestep < STOP)]
#         print (df_test_i)
        X_test_i,y_test_i = shuffle(df_test_i.loc[:,featureColumns],df_test_i['T_self'],random_state=300)

        et_i = ExtraTreesRegressor(n_estimators=n_estimators, n_jobs=-1,random_state=300)
        et_i.fit(X_train_i,y_train_i)
        y_predicted_i = et_i.predict(X_test_i)
#         print ("r2 is: ", r2(y_test_i,y_predicted_i),"mape is: ",mape(y_test_i,y_predicted_i))
        
        temp_y = temp_y.append(y_test_i)
        temp_predicted = np.append(temp_predicted,y_predicted_i)
        
        X_train_i_minus_1 = X_train_i
        X_test_i_minus_1 = X_test_i
        y_train_i_minus_1 = y_train_i
#         y_test_i_minus_1 = y_test_i
        y_predicted_i_minus_1 = y_predicted_i
        
        print('Iteration '+str(i)+' over....')
#         print('\n')
#         if i==2:
#             break

    stop = time()
#     print ('time elapsed for iterative is ',(stop-start),'seconds')
#     print ('\n')

#     print (r2(y_test_i,y_predicted_i) ,mape(y_test_i,y_predicted_i))

    print('\n')
    print ('Iterative Prediction Accuracy for all timesteps predicted: ')
    
#     print (temp_y)
#     print (temp_predicted)
    
#     print (len(temp_y),len(temp_predicted))
    
    print ("r2 is: ", r2(temp_y,temp_predicted) ,"mape is: ", mape(temp_y,temp_predicted))    
    print('\n')
    

    print ('Non-iterative accuracy for last TIME_STEP_ITER timesteps: ')
    df_train_1 = df_big[df_big.timestep < init]
    X_train_1, y_train_1= shuffle(df_train_1.loc[:,featureColumns ], df_train_1['T_self'].values, random_state=300)
    start = time()
    et_direct = ExtraTreesRegressor(n_estimators=n_estimators, n_jobs=-1,random_state=300)
    et_direct.fit(X_train_1,y_train_1)
    y_predicted_direct = et_direct.predict(X_test_i)

    stop = time()
#     print ('time elapsed for direct is ',(stop-start),'seconds')
    print ("r2 is: ", r2(y_test_i,y_predicted_direct), "mape is: ", mape(y_test_i,y_predicted_direct))
    print('\n')

    
    
    print ('Non-iterative accuracy for all timesteps predicted: ')
    
    df_train = df_big[df_big['timestep'] < init]
    df_test = df_big[(df_big['timestep'] >= init) & (df_big['timestep'] < (init+ (TIMESTEP_ITER * n) ))]


    X_train = df_train.loc[:,featureColumns]
    y_train = df_train['T_self']

    X_test = df_test.loc[:,featureColumns]
    y_test = df_test['T_self']

    X_train,y_train = shuffle(X_train,y_train,random_state=300)
    X_test,y_test = shuffle(X_test,y_test,random_state=300)
    
    et_direct = ExtraTreesRegressor(n_estimators=n_estimators, n_jobs=-1,random_state=300)
    et_direct.fit(X_train,y_train)
    predicted = et_direct.predict(X_test)
    print ("r2 is: ", r2(y_test,predicted), "mape is: ", mape(y_test,predicted))

In [13]:
compare_iterative_direct_prediction(n=10,init=200,TIMESTEP_ITER=20,n_estimators=10) # No of iterations, Initial timestep to start prediction, No of timesteps for which temperatures are to be predicted in each interatino, No of estimators in Extra Trees model

Iteration 1 over....
Iteration 2 over....
Iteration 3 over....
Iteration 4 over....
Iteration 5 over....
Iteration 6 over....
Iteration 7 over....
Iteration 8 over....
Iteration 9 over....
Iteration 10 over....


Iterative Prediction Accuracy for all timesteps predicted: 
r2 is:  0.9601 mape is:  2.9672


Non-iterative accuracy for last TIME_STEP_ITER timesteps: 
r2 is:  0.9183 mape is:  3.2508


Non-iterative accuracy for all timesteps predicted: 
r2 is:  0.9614 mape is:  2.4556


In [14]:
compare_iterative_direct_prediction(n=20,init=200,TIMESTEP_ITER=20,n_estimators=10)

Iteration 1 over....
Iteration 2 over....
Iteration 3 over....
Iteration 4 over....
Iteration 5 over....
Iteration 6 over....
Iteration 7 over....
Iteration 8 over....
Iteration 9 over....
Iteration 10 over....
Iteration 11 over....
Iteration 12 over....
Iteration 13 over....
Iteration 14 over....
Iteration 15 over....
Iteration 16 over....
Iteration 17 over....
Iteration 18 over....
Iteration 19 over....
Iteration 20 over....


Iterative Prediction Accuracy for all timesteps predicted: 
r2 is:  0.9562 mape is:  2.6861


Non-iterative accuracy for last TIME_STEP_ITER timesteps: 
r2 is:  0.8914 mape is:  3.9053


Non-iterative accuracy for all timesteps predicted: 
r2 is:  0.9373 mape is:  3.1593


In [15]:
compare_iterative_direct_prediction(n=30,init=200,TIMESTEP_ITER=20,n_estimators=10)

Iteration 1 over....
Iteration 2 over....
Iteration 3 over....
Iteration 4 over....
Iteration 5 over....
Iteration 6 over....
Iteration 7 over....
Iteration 8 over....
Iteration 9 over....
Iteration 10 over....
Iteration 11 over....
Iteration 12 over....
Iteration 13 over....
Iteration 14 over....
Iteration 15 over....
Iteration 16 over....
Iteration 17 over....
Iteration 18 over....
Iteration 19 over....
Iteration 20 over....
Iteration 21 over....
Iteration 22 over....
Iteration 23 over....
Iteration 24 over....
Iteration 25 over....
Iteration 26 over....
Iteration 27 over....
Iteration 28 over....
Iteration 29 over....
Iteration 30 over....


Iterative Prediction Accuracy for all timesteps predicted: 
r2 is:  0.935 mape is:  3.375


Non-iterative accuracy for last TIME_STEP_ITER timesteps: 
r2 is:  0.822 mape is:  4.61


Non-iterative accuracy for all timesteps predicted: 
r2 is:  0.9058 mape is:  3.5953


In [16]:
compare_iterative_direct_prediction(n=40,init=200,TIMESTEP_ITER=20,n_estimators=10)

Iteration 1 over....
Iteration 2 over....
Iteration 3 over....
Iteration 4 over....
Iteration 5 over....
Iteration 6 over....
Iteration 7 over....
Iteration 8 over....
Iteration 9 over....
Iteration 10 over....
Iteration 11 over....
Iteration 12 over....
Iteration 13 over....
Iteration 14 over....
Iteration 15 over....
Iteration 16 over....
Iteration 17 over....
Iteration 18 over....
Iteration 19 over....
Iteration 20 over....
Iteration 21 over....
Iteration 22 over....
Iteration 23 over....
Iteration 24 over....
Iteration 25 over....
Iteration 26 over....
Iteration 27 over....
Iteration 28 over....
Iteration 29 over....
Iteration 30 over....
Iteration 31 over....
Iteration 32 over....
Iteration 33 over....
Iteration 34 over....
Iteration 35 over....
Iteration 36 over....
Iteration 37 over....
Iteration 38 over....
Iteration 39 over....
Iteration 40 over....


Iterative Prediction Accuracy for all timesteps predicted: 
r2 is:  0.9243 mape is:  3.0525


Non-iterative accuracy for last T

In [17]:
compare_iterative_direct_prediction(n=50,init=200,TIMESTEP_ITER=20,n_estimators=10)

Iteration 1 over....
Iteration 2 over....
Iteration 3 over....
Iteration 4 over....
Iteration 5 over....
Iteration 6 over....
Iteration 7 over....
Iteration 8 over....
Iteration 9 over....
Iteration 10 over....
Iteration 11 over....
Iteration 12 over....
Iteration 13 over....
Iteration 14 over....
Iteration 15 over....
Iteration 16 over....
Iteration 17 over....
Iteration 18 over....
Iteration 19 over....
Iteration 20 over....
Iteration 21 over....
Iteration 22 over....
Iteration 23 over....
Iteration 24 over....
Iteration 25 over....
Iteration 26 over....
Iteration 27 over....
Iteration 28 over....
Iteration 29 over....
Iteration 30 over....
Iteration 31 over....
Iteration 32 over....
Iteration 33 over....
Iteration 34 over....
Iteration 35 over....
Iteration 36 over....
Iteration 37 over....
Iteration 38 over....
Iteration 39 over....
Iteration 40 over....
Iteration 41 over....
Iteration 42 over....
Iteration 43 over....
Iteration 44 over....
Iteration 45 over....
Iteration 46 over..